1. Load the Required Libraries

In [1]:
import pandas as pd
import numpy as np
import dtale
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_auc_score,recall_score

C:\Users\Frank\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\dtale\utils.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


2. Read the data from the dataset

In [2]:
df = pd.read_csv("C:\\Users\\Frank\\OneDrive\\Documentos\\ResearchPapers\\Datasets\\diabetes_binary_health_indicators_BRFSS2015.csv", encoding='latin1')
dtale.show(df)

In [ ]:
df.head()
df.drop(columns=['CholCheck','Smoker','Fruits','Veggies','HvyAlcoholConsump','AnyHealthcare','NoDocbcCost','MentHlth','Sex','Education'], inplace=True)

3. Split the attribites into dependent and independent attributes

In [23]:
X = df.iloc[:, 1:].values
Y = df.iloc[:, 0].values
dtale.show(Y, ignore_duplicate=True)

3. Handling missing values and replacing missing values with nan from numpy and replace with mean of all the other values

In [24]:
imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
cols_to_impute= [1,2,3,4,5,6,7,8,9,10]  # Fit the imputer on the data excluding the first column
X[:,cols_to_impute] = imputer.fit_transform(X[:,cols_to_impute])
dtale.show(X, ignore_duplicate=True)

4. Encoding the categorical data

In [ ]:
#cols_to_encode = [3]
# One-hot encode the categorical column
#ct = ColumnTransformer(
   # transformers=[
        ("onehot", OneHotEncoder(), cols_to_encode)  # Correct syntax
    ],
   # remainder="passthrough"
#)
#X = ct.fit_transform(X)  # Returns a sparse matrix (use .toarray() if needed)

# Encode target variable Y (ensure it's 1D)
#labelencoder_Y = LabelEncoder()
#Y = labelencoder_Y.fit_transform(Y.ravel())

5. Splitting the dataset intro training set and test set

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [31]:
X_train.shape

(202944, 11)

6. Perform Feature scaling

In [40]:
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

7. Create a logistic Regression and perform


In [33]:
log_reg = LogisticRegression()
log_reg.fit(X_train, Y_train)
Kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(log_reg, X_train, Y_train, cv=Kfold, scoring='accuracy')


8. Make predictions with the model

In [34]:
y_pred = log_reg.predict(X_test)
log_reg.score(X_test, Y_test)
print(classification_report(Y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.88      0.98      0.93     43739
         1.0       0.53      0.15      0.24      6997

    accuracy                           0.86     50736
   macro avg       0.71      0.57      0.58     50736
weighted avg       0.83      0.86      0.83     50736



In [35]:
log_reg2 = LogisticRegression(max_iter=1000)  # Increase from default 100

8. Retraining the model using Resampled Data


In [36]:
smote_enn = SMOTE(sampling_strategy='minority', random_state=42)
X_train_res, Y_train_res = smote_enn.fit_resample(X_train, Y_train)
print(pd.Series(Y_train_res).value_counts())

0.0    174595
1.0    174595
Name: count, dtype: int64


In [37]:
model = LogisticRegression( random_state= 42)
model.fit(X_train_res, Y_train_res) 
Kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(model, X_train_res, Y_train_res, cv=Kfold, scoring='accuracy')

In [38]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
sensitivity = recall_score(Y_test, y_pred, pos_label=1)
roc_auc = roc_auc_score(Y_test, y_proba)

tn, fp, fn, tp = confusion_matrix(Y_test, y_pred).ravel()
specificity = tn / (tn + fp)

report = classification_report(Y_test, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()

pd.set_option("display.precision", 4)
print(df_report)
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Confusion Matrix:\n{confusion_matrix(Y_test, y_pred)}")
print(f"Mean Accuracy:{scores.mean():.4f} (+/- {scores.std():.4f})")

              precision  recall  f1-score     support
0.0              0.9518  0.7227    0.8216  43739.0000
1.0              0.3080  0.7715    0.4402   6997.0000
accuracy         0.7294  0.7294    0.7294      0.7294
macro avg        0.6299  0.7471    0.6309  50736.0000
weighted avg     0.8631  0.7294    0.7690  50736.0000
ROC-AUC: 0.8220
Sensitivity: 0.7715
Specificity: 0.7227
Confusion Matrix:
[[31609 12130]
 [ 1599  5398]]
Mean Accuracy:0.7441 (+/- 0.0019)
